In [1]:
import sys
sys.path.append('../../../')

In [2]:
import warnings
import pytest
from pathlib import Path
from collections import defaultdict

from fiberphotometry.config import PLOTTING_CONFIG
from fiberphotometry.data.data_loading import DataContainer, load_all_sessions
from fiberphotometry.data.session_loading import populate_containers
from fiberphotometry.data.syncer import sync_session
from fiberphotometry.data.timepoint_processing import create_event_idxs_container_for_sessions
from fiberphotometry.processing.plotting_setup import PlottingSetup
from fiberphotometry.processing.signal_info_setup import assign_sessions_signal_info

In [3]:
baseline_path = Path("../../../Baseline")

In [4]:
sessions = load_all_sessions(
            baseline_dir=str(baseline_path),
            session_type="cpt",
            first_n_dirs=3,
            remove_bad_signal_sessions=True
        );

Processing trial directories:   0%|          | 0/3 [00:00<?, ?it/s]/Users/fsp585/Desktop/gether-demo/FiberphotometryCode/fiberphotometry/notebooks/demos/../../../fiberphotometry/data/data_loading.py:375: UserWarning: Both XLSX and CSV trial‐guide files found for T1 in ../../../Baseline/T1_23.25.29.e. Using 'T1_trial_guide.xlsx'.
  warnings.warn(
Processing trial directories:  33%|███▎      | 1/3 [00:00<00:01,  1.58it/s]/Users/fsp585/Desktop/gether-demo/FiberphotometryCode/fiberphotometry/notebooks/demos/../../../fiberphotometry/data/data_loading.py:375: UserWarning: Both XLSX and CSV trial‐guide files found for T2 in ../../../Baseline/T2_23.25.29.e_2. Using 'T2_trial_guide.xlsx'.
  warnings.warn(
/Users/fsp585/Desktop/gether-demo/FiberphotometryCode/fiberphotometry/notebooks/demos/../../../fiberphotometry/data/data_loading.py:375: UserWarning: Both XLSX and CSV trial‐guide files found for T3 in ../../../Baseline/T3_23.25.29.e_4. Using 'T3_trial_guide.xlsx'.
  warnings.warn(
Processing 

In [5]:
for s in sessions:
    print(s.trial_dir, s.mouse_id)

../../../Baseline/T1_23.25.29.e 23
../../../Baseline/T1_23.25.29.e 25
../../../Baseline/T1_23.25.29.e 29
../../../Baseline/T2_23.25.29.e_2 23
../../../Baseline/T2_23.25.29.e_2 25
../../../Baseline/T2_23.25.29.e_2 29
../../../Baseline/T3_23.25.29.e_4 23
../../../Baseline/T3_23.25.29.e_4 25
../../../Baseline/T3_23.25.29.e_4 29


In [6]:
populate_containers(sessions)

In [7]:
for session in sessions:
    sync_session(session)

Truncating 'phot_470': removed 1 rows
Truncating 'phot_470': removed 1 rows
Truncating 'phot_470': removed 1 rows
Truncating 'phot_470': removed 1 rows
Truncating 'phot_470': removed 1 rows
Truncating 'phot_470': removed 1 rows
Truncating 'phot_470': removed 1 rows


In [8]:
actions_attr_dict = {"Hit": "hit",
                    "Mistake": "mistake", 
                    "Missed Hit": "miss",                    
                    "Correction Trial Correct Rejection": "cor_reject", 
                    "Correct Rejection": "cor_reject"}

reward_attr_dict = {"Reward Collected Start ITI": "reward_collect"}

In [9]:
create_event_idxs_container_for_sessions(sessions, actions_attr_dict, reward_attr_dict)

In [10]:
PlottingSetup(**PLOTTING_CONFIG['cpt']).apply_plotting_setup_to_sessions(sessions)

In [11]:
assign_sessions_signal_info(sessions)

In [12]:
s0 = sessions[0]

In [13]:
for k, v in s0.signal_info.items():
    print(k, v['signal_matrix'].shape)

('DLS', 'G', 'iti_touch') (41, 400)
('DLS', 'G', 'dispimg') (345, 400)
('DLS', 'G', 'hit') (18, 400)
('DLS', 'G', 'mistake') (14, 400)
('DLS', 'G', 'miss') (92, 400)
('DLS', 'G', 'cor_reject') (220, 400)
('DLS', 'G', 'reward_collect') (18, 400)
('DLS', 'G', 'before_dispimg_miss') (92, 400)
('DLS', 'G', 'before_dispimg_cor_reject') (220, 400)
('DLS', 'G', 'before_dispimg_hit') (18, 400)
('DLS', 'G', 'before_dispimg_mistake') (14, 400)
('DLS', 'G', 'after_dispimg_miss') (92, 400)
('DLS', 'G', 'after_dispimg_cor_reject') (220, 400)
('DLS', 'G', 'after_dispimg_hit') (18, 400)
('DLS', 'G', 'after_dispimg_mistake') (14, 400)
('DMS', 'G', 'iti_touch') (41, 400)
('DMS', 'G', 'dispimg') (345, 400)
('DMS', 'G', 'hit') (18, 400)
('DMS', 'G', 'mistake') (14, 400)
('DMS', 'G', 'miss') (92, 400)
('DMS', 'G', 'cor_reject') (220, 400)
('DMS', 'G', 'reward_collect') (18, 400)
('DMS', 'G', 'before_dispimg_miss') (92, 400)
('DMS', 'G', 'before_dispimg_cor_reject') (220, 400)
('DMS', 'G', 'before_dispimg_